In [1]:
import numpy as np
import pandas as pd
import warnings
import json
import requests
import random
import time
from elasticsearch import Elasticsearch
warnings.filterwarnings('ignore')

# 1. Exploračná analýza

## Tabuľka events

In [2]:
events = pd.read_csv('vi_data/events_train.csv')
events.sample(10)

,customer_id,timestamp,event_type,product_id,title,category_name,price
11819545,f1bec958-bcf4-4981-af3f-42df86d70f0a,2019-08-21 17:55:51.593624 UTC,view_item,42a9183d-10c5-4a5d-b2de-e15a0b09128a,Women's bikini top Rip Curl SWIMSUIT INFUSION...,Women>Women's Clothing>Swimmsuits & bikins>Bik...,39.30
7146985,4fdf16d7-2784-49ba-aa69-1f12376f2849,2019-08-03 13:43:48.971047 UTC,view_item,84a504b0-0442-4449-ae4d-583327a571e8,Trendyol Multicolored Patterned Shirt,Women>Women's Clothing>Shirts>Patterned shirts,19.42
11089598,a7130a3a-9eee-4f5d-a9ce-cd28f1f76451,2019-08-18 18:48:49.121503 UTC,view_item,5893415a-0d4f-4e15-b719-a5f5727058f0,LA Gear Full Zip Hoody Ladies,Women>Women's Clothing>Sweatshirts>Zip - Up Sw...,25.65
13600653,5914df58-6929-43f0-8ed6-ac545a65ac5f,2019-08-28 09:36:48.747409 UTC,view_item,3aff8bdb-fb00-439a-bc07-8795a4582c1e,Lonsdale Box T Shirt Mens,Men>Men Clothing>T-Shirts & Polo shirts>Short ...,14.62
9620202,a7506951-70b8-4c30-ade6-827337a04e33,2019-08-12 18:35:54.020504 UTC,view_item,0d1323e1-18dd-49f6-9d9c-0049c8011a66,Nike 'Whatever Print' Long Sleeve Crop Top Ladies,Women>Women's Clothing>T-Shirts>Long Sleeve,29.72
13187086,4726d2dd-87ab-4742-9a33-1b858852c0ac,2019-08-26 20:34:07.699029 UTC,view_item,36348c1a-74ea-470f-ba0d-65209456091f,DKNY Jacket,Men>Men Clothing>Bomber jackets,75.13
6129110,2fbd6e4f-608b-4afb-90f7-af84aec13e99,2019-07-30 10:53:00.657973 UTC,view_item,a1c8b191-afe1-40ee-a48e-730d74acfb2a,Karrimor 3 in 1 Weathertite Jacket Ladies,Sports>Outdoor>Outdoor Clothing>Women's Outdoo...,82.91
4675026,9d1e8669-4e97-4065-8a10-39a0d3914c9c,2019-07-23 15:10:23.347316 UTC,view_item,f47fada5-8871-4cf0-8ebb-fd835edfc8cc,USA Pro Pro Zip Front Swimsuit,Sports>Water Sports>Women's Swimwear >One-piec...,17.97
7115562,a3160c05-d77e-4864-8cb7-99a1adc0bf12,2019-08-03 11:42:46.266079 UTC,view_item,0c9507ad-83d3-4081-a682-0de76547c5f0,Lee Cooper Xlite Hooded Down Jacket Boys,Children>Boy's Clothing>Jackets>Winter jackets,23.52
12653065,4b110f74-2b32-46a9-a0f5-915c58fc9d03,2019-08-25 04:56:06.824595 UTC,view_item,1181d221-a710-4d22-a5bd-0a38010c5acb,Character Padded Coat Infant Girls,Children>Girl's Clothing>Jackets>Winter jackets,38.11


In [102]:
events.shape

(14614385, 7)

In [103]:
events.event_type.unique()

array(['view_item'], dtype=object)

## Tabuľka purchases

In [3]:
purchases = pd.read_csv('vi_data/purchases_train.csv')
purchases.sample(10) 

,customer_id,timestamp,event_type,product_id,title,category_name,price
164291,89e39bab-74a9-40a0-aae9-5f246901ef67,2019-08-24 20:18:36.032272 UTC,purchase_item,860f347d-02cd-43aa-ad10-fdf8d0591eb4,Nike NSW Dress Tee Girls,Children>Girl's Clothing>Dresses & Skirts & Ov...,24.37
104391,9c642b5e-042a-46c2-b956-0c702b781e7f,2019-08-06 07:05:06.639676 UTC,purchase_item,26c9ee01-db1a-437f-86c0-4e60836506b2,Ombre Clothing Men's printed t-shirt S986,Men>Men Clothing>T-Shirts & Polo shirts>Short ...,8.62
128176,6f932bdf-939a-4ead-9859-b998c51f8ebb,2019-08-13 07:04:04.658262 UTC,purchase_item,afa72b2c-e368-4894-81b1-e558fd753526,Women's t-shirt ROXY DARLIN BREAK,Women>Women's Clothing>T-Shirts>Short Sleeve,15.24
146495,adc4c682-4e37-44fa-b203-941dfccbe196,2019-08-19 09:46:56.751796 UTC,purchase_item,71809234-1d68-48e2-a4f6-1fb505213458,Nike Heritage Backpack,Accessories >Backpacks and Bags>Urban Backpacks,22.13
34221,63a08cd5-b37c-47dc-91e6-2af959199871,2019-07-13 16:05:43.488472 UTC,purchase_item,7ef98d44-87e8-4fa8-a5b4-920730244523,Makadamia Woman's Dress M236,NaN,NaN
44013,28ca1707-b444-4a0e-b721-7f4b39161b30,2019-07-16 16:37:41.080111 UTC,purchase_item,7c6b5429-c518-4b66-bff2-eebb978c2445,Women's Tights Gabriella SAVIA,NaN,NaN
179854,0ed39c0f-4313-4cb1-afc0-9f7b6637edde,2019-08-29 09:37:09.860524 UTC,purchase_item,5e8a2e0d-e29b-42a9-b0fb-1e1ab7f1941f,Nike AeroSwift Shorts Ladies,Women's Clothing>Shorts>Women>Sports shorts,55.27
16385,18f77b6f-46e4-45fb-bcab-176f0f631abf,2019-07-07 09:12:30.435546 UTC,purchase_item,e95855d5-fa06-4501-bcc0-b5008c56519b,Lee Cooper Swim Shorts Mens,NaN,NaN
28787,e4158dd6-c44f-427f-b5c2-fdcc8fb6eb96,2019-07-11 17:05:30.733437 UTC,purchase_item,de1d031d-2f64-4560-ab45-58d88b0d6dab,Trendyol Black Halter Embroidered Bikini Top,NaN,NaN
157972,976f088b-e4ff-43fd-a83a-d952cbb03605,2019-08-22 18:28:59.485013 UTC,purchase_item,590ac834-a5db-411a-8cd9-0e353713ed71,Women's dress NUMOCO 217 NEVA,Women>Women's Clothing>Dresses>Mini dress,19.59


In [7]:
purchases.shape

(188712, 7)

In [105]:
purchases.event_type.unique()

array(['purchase_item'], dtype=object)

tabulky events a purchaches majú rovnaké stĺpce 

## Spojenie events and purchases

In [4]:
data = pd.concat([events, purchases], ignore_index=True)
data.sample(10)

,customer_id,timestamp,event_type,product_id,title,category_name,price
3386034,221a5923-57e1-4cc4-a486-d8febeb375b4,2019-07-17 05:35:04.724771 UTC,view_item,8f97ee12-b26d-496f-ad31-7208207321a2,Firetrap Blackseal Bandage Bikini TOP dámsky,NaN,16.90
13065161,30b01c5f-464e-44ab-a3ee-2d5d27a6e397,2019-08-26 13:47:22.3709 UTC,view_item,0daa9625-342c-46c8-b2ad-5db464741124,Trendyol Mint Flywheel Detailed Dress,Women>Women's Clothing>Dresses,23.62
11162283,7cebf3d3-153c-4c14-8cfa-b1c0410ca974,2019-08-19 05:18:56.205214 UTC,view_item,c9efdd08-19c0-4769-a6da-d298159c700b,Lee Cooper Classic Polo Shirt Mens,Men>Men Clothing>T-Shirts & Polo shirts>Polo S...,11.40
14014301,ed4a53ec-a20f-4c08-a935-eafed959b658,2019-08-29 18:42:09.063363 UTC,view_item,88a528ee-d0bc-40c6-a893-734ef8e87538,Lee Cooper Button Down Jacket Mens,Men>Men Clothing>Winter jackets,28.72
9274814,608f85fe-1153-4ba3-b4c1-7d96cf461545,2019-08-11 13:19:37.387294 UTC,view_item,52fbe642-d01d-49d4-b454-5e143754e7ed,Top Secret LADY'S SHOES,Women>Women's Footwear>Sandals>Heeled sandals,22.82
12689240,bbf74481-f80d-4604-b1da-1fd4b2674261,2019-08-25 08:17:34.28507 UTC,view_item,47e26c38-6be9-4222-aaef-4dd3a350cafd,Nike Air Max 90 Jn94,Children>Children's Footwear>Sneakers>Low snea...,119.96
11976239,c7b08491-5dac-409d-8c99-f1ece069bbc7,2019-08-22 10:06:02.876584 UTC,view_item,0c6e3f78-5aff-47e6-8236-064f7297232b,Jack and Jones Marco Chinos,Men>Men Clothing>Pants>Chino pants,45.70
6041944,cc6b97f0-062c-46b0-b13b-d9026256ed81,2019-07-29 22:28:51.383964 UTC,view_item,f3bd844e-a824-4ee0-83c7-b28b4d437a9f,Adidas Barricade 2018 Mens Tennis Shoes,Sports>Tennis>Tennis Shoes>Men's Tennis Shoes,119.60
6004354,f963a627-2a60-467e-8203-da90f696e16d,2019-07-29 19:41:13.58416 UTC,view_item,09211e4d-6c4b-4f7a-b22f-ede5d6c0e61e,Lonsdale Ladbroke Trainers Mens,Men>Men Footwear>Sneakers>Low sneakers,32.58
12801564,bfb905be-24a5-4dd2-9ae1-566ea9216cb7,2019-08-25 15:25:38.550198 UTC,view_item,3100111b-022b-44ff-83b9-f1de6adb20e2,Karrimor Mount Low Ladies Walking Shoes,Sports>Outdoor>Outdoor Shoes>Women's Outdoor S...,62.47


In [5]:
rows_num, cols_num = data.shape
print(rows_num)
print(cols_num)

14803097
7


In [6]:
del events
del purchases

## Atribúty

### customer_id

In [11]:
data.customer_id.describe()

count                                 14803097
unique                                 2211386
top       53f841c7-970b-4e0b-96bf-97a95ecfb734
freq                                      4804
Name: customer_id, dtype: object

In [12]:
len(data[data.customer_id.isna()])

0

In [13]:
rows_num/len(data.customer_id.unique())

6.694035776657716

In [14]:
data.groupby(['customer_id']).count()['timestamp'].median()

2.0

In [14]:
customer_rows = data['customer_id'].value_counts()
customer_rows

53f841c7-970b-4e0b-96bf-97a95ecfb734    4804
9eb76d09-8006-4262-b5eb-459556f338a9    3577
546a1146-64be-4549-a472-a8a4a8ca41b0    3189
a3fe2d55-b8fc-4aaa-8f7f-99f6498b191f    3060
68b2c520-5366-4143-a253-04fd96aeb251    2855
                                        ... 
28f3de37-5d91-451b-9d71-c016ff28765a       1
ddaea6ea-cf1e-4ef3-84d0-4d97f5019dab       1
ee2c4b69-2194-4785-a817-76a30ea8a54b       1
db9f09e8-ea9e-4444-986a-39c973a7dfe3       1
3fe6ea93-4d29-4d6f-990a-6a873f19c2ed       1
Name: customer_id, Length: 2211386, dtype: int64

In [28]:
for i, num in enumerate(customer_rows.value_counts()[:3]):
    print(i+1, num)

1 913643
2 329587
3 199175


In [29]:
(913643)/2211386

0.4131540129131685

In [20]:
(913643+329587)/2211386

0.5621949311427313

In [26]:
for i, num in enumerate(customer_rows.value_counts()[3:50:4]):
    print(4*i+4, num)

4 129721
8 45301
12 22832
16 13311
20 8728
24 6193
28 4481
32 3349
36 2677
40 2124
44 1784
48 1405


- 2,211,386 unikátnych zákazníkov
- žiadne NaN hodnoty
- v priemere vyše 6 záznamov na zákazníka
    - ale median je iba 2
    - 913,643 (41%) zákazníkov má len 1 záznam
    - 56% zákazníkov má 2 záznamy

### timestamp

In [16]:
data.timestamp.describe()

count                           14803097
unique                          14802889
top       2019-07-22 04:33:52.889148 UTC
freq                                   4
Name: timestamp, dtype: object

In [17]:
data.loc[data['timestamp'] == "2019-07-22 04:33:52.889148 UTC"]

,customer_id,timestamp,event_type,product_id,title,category_name,price
4361630,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,5bbe88b4-9650-4c1a-8f95-9a09c264b2c9,Hot Tuna Crew T Shirt Mens,Men>Men Clothing>T-Shirts & Polo shirts,12.84
4361631,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,248faf65-d1e2-4d7b-84c8-ce1c459ee086,Lonsdale Jersey Polo Shirt Mens,Men>Men Clothing>T-Shirts & Polo shirts>Polo S...,12.90
4361632,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,377bdb75-6f46-4c29-b559-9af0c6ba2ae8,Pierre Cardin Web Belt Shorts Mens,Men>Men Clothing>Shorts,17.12
4361633,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,b2bcef2f-df2d-4c23-aee7-73ca112ef815,Lee Cooper Men's boxers 5-pack,Men>Men Clothing>Underwear>Boxer shorts & trunks,13.10


In [18]:
min(data.timestamp)

'2019-07-01 00:00:01.101977 UTC'

In [19]:
max(data.timestamp)

'2019-08-31 23:59:59.536067 UTC'

In [7]:
len(data[data.timestamp.isna()])

0

- dátumy z intervalu 2 mesiacov
- žiadne NaN hodnoty
- existujú záznamy typu `view_item`, kde je daný zákazník v danom čase a rôzne produkty. 

### event_type

In [21]:
data.event_type.unique()

array(['view_item', 'purchase_item'], dtype=object)

In [22]:
len(data[data.event_type.isna()])

0

In [23]:
len(data.loc[data['event_type'] == "view_item"])/rows_num

0.9872518568242848

In [24]:
len(data.loc[data['event_type'] == "purchase_item"])/rows_num

0.01274814317571519

- dva typy: `view_item` a `purchase_item`
- pomer je 98.7% ku 1.27%
- žiadne NaN hodnoty

### product_id

In [25]:
data.product_id.describe()

count                                 14803097
unique                                  123837
top       31ae211d-57a9-4bfa-b95f-40776af03c3f
freq                                     52736
Name: product_id, dtype: object

In [60]:
len(data[data.product_id.isna()])

0

In [61]:
rows_num/len(data.product_id.unique())

119.53694776197744

In [62]:
data.groupby(['product_id']).count()['timestamp'].median() 

17.0

In [32]:
product_rows = data['product_id'].value_counts()
product_rows

31ae211d-57a9-4bfa-b95f-40776af03c3f    52736
0f943312-7141-4606-abfa-81fd63a5498f    49141
a8f0292d-5fca-42b9-b0d3-b38e7efa416b    48343
e33cea18-ee3f-4e37-83a7-1d88c54d83b8    45421
608f32cd-210c-4af5-99d8-6182a9678cf5    39801
                                        ...  
6b0e71d0-cdca-4160-8df1-ae7b43ef967c        1
630d92d1-ed2b-4a75-a778-3dacba195eb5        1
0633f706-85d4-4ac6-b34d-7f653fa7139d        1
a3c46419-d8fb-4273-88ba-06dc92b42696        1
326e712f-9733-4761-b458-217f3f695833        1
Name: product_id, Length: 123837, dtype: int64

In [36]:
for i, num in enumerate(product_rows.value_counts()[:50:4]):
    print(4*i+1, num)

1 15982
5 3582
9 2365
13 1792
17 1376
21 1139
25 948
29 810
33 694
37 624
41 528
45 501
49 436


- 123,837 unikátnych produktov
- žiadne NaN hodnoty
- v priemere vyše 119 záznamov na produkt
    - maximum je 52,736
    - minimum je 1
    - median je 17
    - 15,982 produktov má len 1 záznam

### title

In [63]:
data.title.describe()

count                     14801284
unique                       71440
top       Lee Cooper Marl Backpack
freq                        168766
Name: title, dtype: object

In [72]:
data.loc[data['product_id'] == "31ae211d-57a9-4bfa-b95f-40776af03c3f"].title.unique()

array(['Batoh Lee Cooper Marl', 'Lee Cooper Marl Backpack'], dtype=object)

In [64]:
len(data[data.product_id.isna()])

0

- 71,440 unikátnych označení (našli sme 123,837 unikátnych produktov)
    - produkt môže mať viac rôznych označení
- žiadne NaN hodnoty

### price

In [85]:
data.price.describe().apply(lambda x: format(x, 'f'))

count    14758211.000000
mean           33.122065
std           121.532833
min             0.060000
25%            12.710000
50%            21.860000
75%            39.490000
max         32636.460000
Name: price, dtype: object

In [87]:
np.nanmedian(data.price)

21.86

In [73]:
len(data[data.price.isna()])

44886

In [89]:
data.loc[data['product_id'] == "31ae211d-57a9-4bfa-b95f-40776af03c3f"].price.unique()

array([12.9 , 12.73, 13.11, 13.32, 12.49, 12.71, 13.45, 12.65, 12.83,
       11.46, 11.8 , 11.44, 11.38, 12.1 , 11.24, 11.61, 11.99, 11.55,
        9.55,  9.56, 10.61, 10.63, 13.49,  7.32, 12.14, 12.27, 12.33,
       13.04, 12.92, 12.51, 12.11, 12.45, 12.35, 13.09, 16.84, 16.41,
       16.11, 16.03, 16.23, 15.95, 16.  , 15.58, 16.39, 16.97, 13.65,
       14.75, 13.98, 14.02, 14.05, 14.22, 14.38, 14.87, 14.12, 14.37,
       12.46, 12.37, 13.12, 12.04, 13.02, 12.39, 12.68, 12.55, 12.69,
         nan])

- priemer ceny je cca 33
- median ceny je cca 22
- 44,886 NaN hodnôt
- jeden produkt môže mať viac cien

### category_name

In [90]:
data.category_name.describe()

count                                   11367365
unique                                       713
top       Men>Men Footwear>Sneakers>Low sneakers
freq                                      462179
Name: category_name, dtype: object

In [103]:
for pid in data.product_id.sample(10):
    unique_categories = data.loc[data['product_id'] == pid].category_name.unique()
    print(len(unique_categories), unique_categories)

2 [nan 'Men>Men Clothing>T-Shirts & Polo shirts>Polo Shirts']
3 [nan "Women>Women's Clothing>Shirts"
 "Women>Women's Clothing>Shirts>Patterned shirts"]
2 [nan "Women>Women's Clothing>Jackets & Coats>Coats"]
2 [nan "Women>Women's Footwear>Slip-on"]
2 [nan "Women>Women's Clothing>Dresses>Wedding dress"]
2 [nan 'Men>Men Clothing>T-Shirts & Polo shirts>Polo Shirts']
2 [nan "Sports>Running>Running Shoes>Women's Running Shoes"]
2 [nan 'Accessories >Backpacks and Bags>Fashion backpacks']
3 ["Women>Women's Clothing>Pants & Jeans>Pants"
 "Women>Women's Clothing>Pants >Pants"
 "Women>Women's Clothing>Pants >Single-colored pants"]
2 [nan "Women>Women's Clothing>Jumpsuits"]


In [96]:
category_name_nans_num = len(data[data.category_name.isna()])
category_name_nans_num

3435732

In [95]:
category_name_nans_num/rows_num

0.23209548650529008

In [45]:
data["category_name"].str.split(">", expand = True).shape

(14803097, 7)

In [40]:
data_15K = data.sample(15000)

In [116]:
categories = data_15K["category_name"].str.split(">", expand = True)

Keďže `data` má skoro 15M záznamov, spracovanie trvá veľmi dlho. Vybrali sme len 15K.

In [123]:
categories.sample(5)

,0,1,2,3,4
2563389,NaN,NaN,NaN,NaN,NaN
7189282,Women,Women's Footwear,Balerinas,None,None
11478593,Men,Men Clothing,T-Shirts & Polo shirts,Short Sleeve,None
6003198,Women,Women's Clothing,Swimmsuits & bikins,One-piece,None
1495798,NaN,None,None,None,None


In [136]:
for i in range(5):
    print('col_{} unique values ='.format(i), len(categories[i].unique()))
    print('col_{} NaN values ='.format(i), len(categories[categories[i].isna()]))
    print()

col_0 unique values = 21
col_0 NaN values = 3455

col_1 unique values = 38
col_1 NaN values = 3551

col_2 unique values = 154
col_2 NaN values = 3669

col_3 unique values = 216
col_3 NaN values = 6845

col_4 unique values = 49
col_4 NaN values = 14513



In [126]:
categories[0].unique()

array([nan, 'Men', 'Women', 'Sports', "Women's Clothing", 'Children',
       'Accessories ', 'Backpacks and Bags', 'Men Clothing',
       "Boy's Clothing", 'Sale - Men', 'Sale - Women', 'Dresses',
       'NEW IN', 'Unlisted', "Children's Footwear", 'strom kategorii',
       'T-Shirts', 'NY FASHION WEEK', 'Sale - Children',
       'Delivery Immediately'], dtype=object)

- **iba** 713 unikátnych kategórií (našli sme 123,837 unikátnych produktov)
- jeden produkt môže mať rôzne `category_name` hodnoty, vrátane NaN
- 23.2% záznamov je NaN

- po rozparsovaní stromu `category_name` vzniklo 7 stĺpcov (t.j. výška stromu)
- smerom od koreňa rastie počet NaN hodnôt a unikátnych hodnôt
- vyskytujú sa tématicky rovnaké len inak napísané kategórie


### Diskusia
- Produkty zákazníka sa môžu opakovať, a preto je vhodné odporúčať už pozerané produkty
- Potreba zjednotiť nejednotné atribúty rovnakého produktu:
    - `title`
    - `category_name`
    - možno `cena`

---

# 2. Zjednotenie a spracovanie dát

In [7]:
categories = data["category_name"].str.split(">", expand = True)
del data["category_name"]
category_names = {}
for i in range(7):
    category_names[i] = "category_{}".format(i) 
categories = categories.rename(columns=category_names)
data = pd.concat([data, categories], axis=1, sort=False)
del categories

Vyparsovanie `category`

In [88]:
# Uloženie medzi-výsledku
# data.to_csv('vi_data/data_train.csv', encoding='utf-8')
# data = pd.read_csv('vi_data/data_train.csv', index_col=0)

In [219]:
data.sample(3)

,customer_id,timestamp,event_type,product_id,title,price,category_0,category_1,category_2,category_3,category_4,category_5,category_6
13029308,a307441c-2131-4cfe-a3f5-ceba85324cdb,2019-08-26 11:38:54.520998 UTC,view_item,7e40885a-c1e4-48e3-80e3-71951fc25a44,Children's Ski jacket LOAP FALDA,39.52,Children,Boy's Clothing,Jackets,Winter jackets,NaN,NaN,NaN
9634041,0e42ec0e-8d82-4d4a-b6ef-17ef06e3de1a,2019-08-12 19:25:21.494362 UTC,view_item,43bee0d7-2d3f-42ac-bef2-99de2b660be2,Top Secret LADY'S DRESS,15.34,Women,Women's Clothing,Dresses,Mini dress,NaN,NaN,NaN
14697110,7c04bdf3-0ea1-4830-9047-0350d7071ef2,2019-07-31 09:10:17.624419 UTC,purchase_item,ca3ccb9a-933f-45fb-a10a-dfde4eb046ee,Everlast Heavy Bomber Jacket Mens,35.00,Men Clothing,Men,Jackets & Coats,Bomber jackets,NaN,NaN,NaN


## Vytvorenie elastic indexu a naplnenie produktami

**PUT http://localhost:9200/vi_products**
```json
{'settings': {'number_of_shards': 1, 'number_of_replicas': 0},
 'mappings': {'properties': {'product_id': {'type': 'keyword'},
   'title': {'type': 'text', 'analyzer': 'english'},
   'price': {'type': 'double'},
   'events_number': {'type': 'integer'},
   'category_0': {'type': 'keyword'},
   'category_1': {'type': 'keyword'},
   'category_2': {'type': 'keyword'},
   'category_3': {'type': 'keyword'},
   'category_4': {'type': 'keyword'},
   'category_5': {'type': 'keyword'},
   'category_6': {'type': 'keyword'}}}}
```

In [194]:
product_ids = list(data.product_id.unique())

In [195]:
len(product_ids)

123837

In [192]:
def fill_index():
    es = Elasticsearch([{'host': '127.0.0.1', 'port': 9200}])
    doc = {}
    for pid in product_ids:
        rows = data[data.product_id == pid]
        doc['product_id'] = pid
        try:
            doc['title'] = list(rows.title.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['price'] = np.mean(rows.price.dropna().unique())
        except IndexError:
            pass
        try:
            doc['events_number'] = len(rows)
        except IndexError:
            pass
        try:
            doc['category_0'] = list(rows.category_0.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_1'] = list(rows.category_1.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_2'] = list(rows.category_2.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_3'] = list(rows.category_3.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_4'] = list(rows.category_4.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_5'] = list(rows.category_5.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_6'] = list(rows.category_6.dropna().unique())[0]
        except IndexError:
            pass
        if np.isnan(doc['price']):
            del doc['price']
        es.index(index='vi_products', body=doc)

In [218]:
# fill_index()

---

# 3. Obsahový odporúčač

Navrhli sme obsahový odporúčač, ktorý používa nástroj Elasticsearch a dopyt [More Like This](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-mlt-query.html).

In [101]:
def ES_recommender_should(product_ids, fields=["title", "category_0", "category_1", "category_2", "category_3"], boosting_recent=False, count=10):
    """
    Using elasticsearch more_like_this query with should clause
    """
    url = 'http://localhost:9200/vi_products/_search'
    def transform_id(pid):
        data = {'query': {'bool': {'filter': {'term': {'product_id': pid}}}}}
        r = requests.post(url, json=data)
        try:
            return json.loads(r.text)['hits']['hits'][0]['_id']
        except IndexError:
            return None
    
    ids = [transform_id(pid) for pid in product_ids if pid]
    should = []
    for i, _id in enumerate(ids):
        boost = i if boosting_recent else 1
        if _id:
            mlt = {
              "more_like_this": {
                "fields": fields,
                "like": [
                  {
                    "_index": "vi_products",
                    "_id": _id
                  }
                ],
                "min_term_freq": 1,
                "min_doc_freq": 1,
                "max_query_terms": 100,
                "include": True,
                "boost": boost
              }
            }
            should.append(mlt)
            
    data = {
              "from": 0,
              "size": 10,
              "query": {
                "bool": {
                  "should": should
                }
              }
            }    
#     print(json.dumps(data))
    r = requests.post(url, json=data)
    try: 
        return [hit['_source']['product_id'] for hit in json.loads(r.text)['hits']['hits']]
    except KeyError as e:
        print('KeyError:'+e, r.text) 

Odporúča produkty, ktoré sú v súčte najpodobnejšie všetkým z trénovacím produtov

In [22]:
def ES_recommender_dis_max(product_ids, fields=["title", "category_0", "category_1", "category_2", "category_3"], boosting_recent=False, count=10):
    """
    Using elasticsearch more_like_this query with dis_max clause
    """
    url = 'http://localhost:9200/vi_products/_search'
    def transform_id(pid):
        data = {'query': {'bool': {'filter': {'term': {'product_id': pid}}}}}
        r = requests.post(url, json=data)
        try:
            return json.loads(r.text)['hits']['hits'][0]['_id']
        except IndexError:
            return None
    
    ids = [transform_id(pid) for pid in product_ids if pid]
    queries = []
    for i, _id in enumerate(ids):
        boost = i if boosting_recent else 1
        if _id:
            mlt = {
              "more_like_this": {
                "fields": fields,
                "like": [
                  {
                    "_index": "vi_products",
                    "_id": _id
                  }
                ],
                "min_term_freq": 1,
                "min_doc_freq": 1,
                "max_query_terms": 100,
                "include": True,
                "boost": boost
              }
            }
            queries.append(mlt)
            
    data = {
              "from": 0,
              "size": 10,
              "query": {
                "script_score": {
                  "query": {
                    "dis_max": {
                      "queries": queries
                    }
                  },
                  "script": {
                    "source": "(Math.log(doc['events_number'].value) / Math.log(10)) * _score"
                  }
                }
              }
            }   
#     print(json.dumps(data))
    r = requests.post(url, json=data)
    try: 
        return [hit['_source']['product_id'] for hit in json.loads(r.text)['hits']['hits']]
    except KeyError as e:
        print('KeyError:'+e, r.text) 

Odporúča produkty, ktoré sú najpodobnejšie niektorému z trénovacích produtov

# 4. Evaluácia

## Metriky

In [57]:
def precision_at_k(r, k=10):
    """Score is precision @ k
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Precision @ k
    Raises:
        ValueError: len(r) must be > 0
    """
    k = min(k, len(r))
    assert k > 0
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def dcg_at_k(r, k=10):
    """Score is discounted cumulative gain (dcg)
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
    Returns:
        Discounted cumulative gain
    Raises:
        ValueError: len(r) must be > 0
    """
    k = min(k, len(r))
    assert k > 0
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.

def ndcg_at_k(r, k=10):
    """Score is normalized discounted cumulative gain (ndcg)
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
    Returns:
        Normalized discounted cumulative gain
    Raises:
        ValueError: len(r) must be > 0
    """
    k = min(k, len(r))
    assert k > 0
    idcg = dcg_at_k(sorted(r, reverse=True), k)
    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

Upravené z [Ranking Metrics](https://gist.github.com/bwhite/3726239)

In [64]:
def evaluate(customer_id, recomender_function, metric_function):
    """
    Split customer's product, call recomender_function, compure relevance and metric_function
    """
    def split_products(products):
        middle = (len(products)+1)//2
        return [products[:middle], products[middle:]]
    customer_products = events[events.customer_id == customer_id].sort_values(by ='timestamp')
    if len(customer_products) == 1:
        raise Exception(customer_id + ' has just one product')
    product_ids = list(customer_products.product_id)
    train, test = split_products(product_ids)
    recomended = recomender_function(train, 
                                     fields=["title", "category_0", "category_1", "category_2", "category_3", "category_4", "category_5", "category_6"], 
                                     boosting_recent=True, 
                                     count=10)
    r = list()
    k = min(len(test), len(recomended))
    test = test[:k]
    recomended = recomended[:k]
    for i in range(k):
        r.append(1 if recomended[i] in test else 0)
    rank = metric_function(r)
    return rank #[train, test, recomended, r, rank]

Na vygenerovenie odporúčania bola použitá tabuľka `events`

## Výsledky

In [9]:
customer_ids = sorted(list(events.customer_id.unique()), key=lambda k: random.random())

In [58]:
len(customer_ids)

2210171

In [59]:
def results(recomender_function, metric_function, NUMBER_TESTED=10):
    start = time.time()
    p = []
    for cid in customer_ids:
        try:
            p.append(evaluate(cid, recomender_function, metric_function))
        except:
            pass
        if len(p) >= NUMBER_TESTED:
            break
    print('seconds:', time.time()-start)
    print('type:', recomender_function.__name__)
    print('number of tested customers:', len(p))
    print('average', metric_function.__name__,':', np.mean(p))

In [60]:
results(ES_recommender_should, ndcg_at_k, 300) #ES_recommender_should, title + 4 categoties

seconds: 484.64383912086487
type: ES_recommender_should
number of tested customers: 300
average ndcg_at_k : 0.2508594652179127


In [61]:
results(ES_recommender_dis_max, ndcg_at_k, 300) #ES_recommender_dis_max, title + 4 categoties

seconds: 691.1087830066681
type: ES_recommender_dis_max
number of tested customers: 300
average ndcg_at_k : 0.327475554946516


In [63]:
results(ES_recommender_dis_max, ndcg_at_k, 300) #ES_recommender_dis_max, title + all categoties

seconds: 622.2124481201172
type: ES_recommender_dis_max
number of tested customers: 300
average ndcg_at_k : 0.36574870105544943


In [70]:
results(ES_recommender_dis_max, ndcg_at_k, 300) #ES_recommender_dis_max, title + all categoties + boosting_recent

seconds: 710.6878938674927
type: ES_recommender_dis_max
number of tested customers: 300
average ndcg_at_k : 0.403434364394986


| **method**                                                       | **ndcg_at_10, average(300)** |
|------------------------------------------------------------------|------------------------------|
| ES_recommender_should, title + 4 categoties                      | 0.25085                      |
| ES_recommender_dis_max, title + 4 categoties                     | 0.32747                      |
| ES_recommender_dis_max, title + all categoties                   | 0.36574                      |
| ES_recommender_dis_max, title + all categoties + boosting_recent | 0.40343                      |

# 5. vi-challenge kaggle

In [14]:
vi_challenge_uID = pd.read_csv('vi_challenge_uID.csv', header=None)
len(vi_challenge_uID)

1000

In [23]:
start = time.time()
output = list()
most_popular_products = list(data.product_id.value_counts().index[:10])
for i, uid in enumerate(vi_challenge_uID[0]):
    customer_products = data[data.customer_id == uid].sort_values(by ='timestamp')
    product_ids = list(customer_products.product_id)
    if product_ids:
        recomended = ES_recommender_dis_max(product_ids,
                                            fields=["title", "category_0", "category_1", "category_2", "category_3", "category_4", "category_5", "category_6"], 
                                            boosting_recent=False, 
                                            count=10)
        print(i, ": ", len(recomended))
        for r in recomended:
            output.append([uid, r])
    else:
        print(i, "MPP")
        for r in most_popular_products:
            output.append([uid, r])
print('seconds:', time.time()-start)

0 :  10
1 :  10
2 MPP
3 MPP
4 :  10
5 MPP
6 :  10
7 MPP
8 :  10
9 :  10
10 MPP
11 :  10
12 :  10
13 :  10
14 :  10
15 :  10
16 :  10
17 MPP
18 MPP
19 MPP
20 MPP
21 :  10
22 :  10
23 :  10
24 :  10
25 :  10
26 :  10
27 :  10
28 MPP
29 :  10
30 :  10
31 :  10
32 :  10
33 MPP
34 :  10
35 :  10
36 MPP
37 :  10
38 MPP
39 :  10
40 :  10
41 :  10
42 :  10
43 :  10
44 :  10
45 :  10
46 :  10
47 :  10
48 :  10
49 :  10
50 :  10
51 :  10
52 MPP
53 :  10
54 :  10
55 :  10
56 :  10
57 :  10
58 :  10
59 :  10
60 :  10
61 :  10
62 :  10
63 :  10
64 :  10
65 :  10
66 :  10
67 :  10
68 :  10
69 :  10
70 :  10
71 :  10
72 :  10
73 :  10
74 :  10
75 :  10
76 :  10
77 MPP
78 :  10
79 :  10
80 :  10
81 :  10
82 :  10
83 :  10
84 MPP
85 MPP
86 :  10
87 :  10
88 MPP
89 :  10
90 :  10
91 :  10
92 :  10
93 :  10
94 :  10
95 :  10
96 :  10
97 :  10
98 :  10
99 :  10
100 :  10
101 :  10
102 :  10
103 :  10
104 :  10
105 :  10
106 :  10
107 :  10
108 :  10
109 :  10
110 :  10
111 :  10
112 :  10
113 :  10
114 : 

856 :  10
857 :  10
858 :  10
859 MPP
860 :  10
861 MPP
862 :  10
863 :  10
864 :  10
865 :  10
866 :  10
867 :  10
868 :  10
869 :  10
870 :  10
871 :  10
872 MPP
873 :  10
874 :  10
875 :  10
876 :  10
877 :  10
878 MPP
879 :  10
880 :  10
881 :  10
882 :  10
883 :  10
884 MPP
885 :  10
886 :  10
887 :  10
888 :  10
889 :  10
890 :  10
891 :  10
892 :  10
893 :  10
894 :  10
895 :  10
896 :  10
897 :  10
898 :  10
899 :  10
900 :  10
901 :  10
902 MPP
903 :  10
904 :  10
905 :  10
906 :  10
907 :  10
908 :  10
909 :  10
910 :  10
911 :  10
912 MPP
913 :  10
914 :  10
915 MPP
916 :  10
917 MPP
918 :  10
919 MPP
920 :  10
921 :  10
922 :  10
923 :  10
924 :  10
925 :  10
926 :  10
927 :  10
928 :  10
929 :  10
930 MPP
931 :  10
932 MPP
933 :  10
934 :  10
935 :  10
936 :  10
937 :  10
938 :  10
939 :  10
940 MPP
941 :  10
942 :  10
943 :  10
944 MPP
945 MPP
946 :  10
947 :  10
948 :  10
949 MPP
950 MPP
951 :  10
952 :  10
953 :  10
954 :  10
955 :  10
956 MPP
957 :  10
958 :  10
959 : 

In [24]:
submission = pd.DataFrame(output, columns=['customer_id', 'product_id'])
submission.to_csv('harnusekSubmission.csv', index=False)
len(submission)

10000

##### [Naše najlepšie skóre v súťaži:  0.10909](https://www.kaggle.com/harnusek)
- použitie `ES_recommender_dis_max, title + all categoties`

# 6. Zhodnotenie

1. Počas exploračnej analýzy datasetu sme vypočítali štatistické údaje pre každý stĺpec a identifikovali sme potrebu ďalšieho predspracovania

2. Následne sme vyparsovali stĺpec `category` na 7 stĺpcov/kategórií, zjednotli sme atribúty rovnakých produktov a vložili všetky produkty do Elasticsearch indexu

3. Navrhli sme obsahový odporúčač, ktorý používa Elasticsearch dopyt [More Like This](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-mlt-query.html)
    - vytvorili sme 2 varianty odporúčača:
        - `ES_recommender_should`: odporúča produkty, ktoré sú v súčte najpodobnejšie všetkým z trénovacím produtov
        - `ES_recommender_dis_max`: odporúča produkty, ktoré sú najpodobnejšie niektorému z trénovacích produtov
    - ich vstupom sú mená stĺpce na základe ktorých sa určuje podobnosť 
    - je možné použiť "boostovanie" podobnosti produktov od najnovšie pozeraných po najstaršie (`boosting_recent`)
    
4. Počas evaluácie sme porovnávali oba viarianty a viaceré nastavenia parametrov s použitím metrík `precision_at_k` a `ndcg_at_k`
    - Podľa našich testov je najúspešnejší `ES_recommender_dis_max` s použitím `title`, všetkými kategóriami a `boosting_recent`
5. V kaggle súťaži sme dosiahli najlepšie skóre: [0.10909](https://www.kaggle.com/harnusek)
    - použitie `ES_recommender_dis_max` s použitím `title` a všetkými kategóriami

## Výhody riešenia
- vygenerovanie odporúčaní je pomerne rýchle (1000 zákazníkov trvá cca 20 minút)
- naše riešenie umožnuje jednoduché pridanie ďalších produktov, bez nutnosti pretrénovania
- možnosť jednoduchého experimentovania s parametrami

## Nevýhody riešenia
- zjednotenie dát a naplnenie indexu bolo veľmi pomalé (>24hodín)
- použitie len prístupu založenom na obsahu (v budúcnosti: možnosť použitia popularity produktov `events_number` v indexe)
- nepoužitie cien produktov